In [1]:
import pandas as pd
import numpy as np

In [5]:
%%time
import funcs

rats_gdo = funcs.get_rats_gdo()
rats_gdo.head(3)

CPU times: user 2.49 s, sys: 46.7 ms, total: 2.54 s
Wall time: 3.11 s


,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,NOM_OPERACAO,...,CIA,VIATURAS,EFETIVO,EFETIVIDADE_PARCIAL,Qde de locais fiscalizados,Qde de pessoas abordadas,Qde de pessoas que sopraram o etilometro,Qde de veiculos fiscalizados,EFICIENCIA_E_EFICACIA,EFETIVIDADE
RAT.NUM_ATIVIDADE,,,,,,,,,,,,,,,,,,,,,
2018-046472984-001,Y07001,OPERACAO DE BATIDA POLICIAL,17/10/2018,17/10/2018,18:43,17/10/2018,19:00,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,139 CIA,1,2,0,0,0,0,0,0,0
2018-051709149-001,Y07001,OPERACAO DE BATIDA POLICIAL,20/11/2018,20/11/2018,05:25,20/11/2018,07:15,TRANSEUNTE,METALURGICA / SIDERURGICA,,...,142 CIA,1,2,0,0,0,0,0,0,0
2018-052156427-001,Y07001,OPERACAO DE BATIDA POLICIAL,22/11/2018,22/11/2018,20:20,22/11/2018,20:45,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,142 CIA,1,2,0,0,0,0,0,0,0


<h2 style='color:#009'>Código para gerar relatório de LEI SECA, por Cias, por ano, por mês</h2>

In [6]:
rats_ols = rats_gdo[        
    (~rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('PRV'))
    & (rats_gdo['NAT.CODIGO'] == 'Y04012')
]
rats_ols.loc[:,'CIA_OLS'] = np.select([
    rats_ols.loc[:,'NOM_UNID_RESPONSAVEL'].str.contains('240')
], [
    '240 CIA'
    ], default = rats_ols.loc[:,'CIA'])

ols_by_cia = rats_ols[
    rats_ols['ANO'] >= 2020
].groupby(['ANO','MES','CIA_OLS']).sum()['EFETIVIDADE'].unstack(['MES']).fillna(0)

ols_by_cia.loc[:,'ACUM'] = ols_by_cia.sum(1)
ols_by_cia.loc[(2020, '23 BPM'),:] = ols_by_cia.sum()

# rats_ols

ols_by_cia = ols_by_cia.astype('int16')
ols_by_cia.columns =  pd.MultiIndex.from_product([['OLS'], ols_by_cia.columns])
ols_by_cia

OLS                  
               1  2  3   4  5 ACUM
ANO  CIA_OLS                      
2020 139 CIA   1  0  1   2  0    4
     142 CIA   0  1  0   0  0    1
     240 CIA   0  1  2   6  0    9
     51 CIA    0  3  1   2  0    6
     53 CIA    2  0  0   1  0    3
     23 BPM    3  5  4  11  0   23

<h2 style='color:#009'>Código para gerar relatório de RQV, por setores, por ano, por mês</h2>

In [7]:
rats_rqv = rats_gdo[
    (rats_gdo['NAT.CODIGO'].str.contains('Y0700[1345]')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('IND PE')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('BPE')) &
    ( ~ rats_gdo['NOM_UNID_RESPONSAVEL'].str.contains('CPE')) &
    (rats_gdo['ANO'] == 2020)
]

rqv_by_cia = rats_rqv.groupby(['MES','CIA']).sum()[['EFICIENCIA_E_EFICACIA','EFETIVIDADE']]
rqv_by_cia = rqv_by_cia.unstack('MES')
rqv_by_cia.loc['23 BPM'] = rqv_by_cia.sum()
rqv_by_cia.loc[:,('EFICIENCIA_E_EFICACIA', 'ACUM')] = rqv_by_cia.loc[:,'EFICIENCIA_E_EFICACIA'].sum(axis=1)
rqv_by_cia.loc[:,('EFETIVIDADE', 'ACUM')] = rqv_by_cia.loc[:,'EFETIVIDADE'].sum(axis=1)
rqv_by_cia = rqv_by_cia[['EFICIENCIA_E_EFICACIA', 'EFETIVIDADE']]
rqv_by_cia

EFICIENCIA_E_EFICACIA                           EFETIVIDADE            \
MES                         1    2    3    4    5  ACUM           1    2    3   
CIA                                                                             
139 CIA                   214  235  208  154   61   872          44   40   37   
142 CIA                   416  369  337  205   65  1392         142  143  138   
51 CIA                    251  154  126  120   59   710          92   59   63   
53 CIA                    221  175  188  149   54   787          54   30   27   
23 BPM                   1102  933  859  628  239  3761         332  272  265   

                        
MES        4   5  ACUM  
CIA                     
139 CIA   34  17   172  
142 CIA   84  31   538  
51 CIA    50  33   297  
53 CIA    23   7   141  
23 BPM   191  88  1148

In [23]:
writer = pd.ExcelWriter('rqv_e_ols.xlsx', engine='xlsxwriter')
rqv_by_cia.to_excel(writer, sheet_name='RQV')
ols_by_cia.to_excel(writer, sheet_name='OLS')
rats_gdo.to_excel(writer, sheet_name='Todos os registros')
writer.save()